In [ ]:
#| export
import numpy as np
import torch
import pickle
import io
import torch
from copy import deepcopy
from fastcore.all import *
import matplotlib.pyplot as plt

In [ ]:
#| default_exp utils

## Training utilities

In [ ]:
#| export
def plot_trainings(training_results, eval_optims=True, expl_optims=False, figsize=(10, 5),
                   fontsize=18, ticksize=14):
    "Plots the training results."
    use_legend = eval_optims or expl_optims
    mean_rewards = np.mean(training_results['rewards'], axis=0)
    std_rewards = np.std(training_results['rewards'], axis=0)
    mean_costs = np.mean(training_results['costs'], axis=0)

    fig, ax1 = plt.subplots(figsize=figsize)
    color1 = "tab:blue"
    ax1.set_xlabel("Training episodes", fontsize=fontsize)
    ax1.set_ylabel("Reward", color=color1, fontsize=fontsize)
    ax1.plot(mean_rewards, alpha=0.8, label="Rewards")
    ax1.set_ylim([-0.05, 1.05]); ax1.set_yticks(np.arange(11, step=2)/10)
    if eval_optims: 
        ax1.plot(np.mean(training_results['eval_optims'], axis=0), alpha=0.8, color='C3',
                 label="Optims eval")
    if expl_optims: 
        ax1.plot(np.mean(training_results['expl_optims'], axis=0), alpha=0.8, color='C4',
                 label="Optims expl")

    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.tick_params(labelsize=ticksize)
    ax1.grid()
    if use_legend: plt.legend(loc='best', fontsize=ticksize)

    ax2 = ax1.twinx()
    color2 = "tab:orange"
    ax2.set_ylabel("Cost",color=color2,fontsize=fontsize)
    ax2.plot(mean_costs,color=color2,alpha=0.5,label="Cost")
    ax2.tick_params(axis='y', labelcolor=color2, labelsize=ticksize)

    fig.tight_layout()
    
def arrange_shape(obj, mode='expand'):
    "Makes all lists within a list have the same length. Can be mode 'cut' or 'expand'"
    rows = len(obj)
    if mode == 'expand':
        cols = max([len(obj[k]) for k in range(rows)])
        M = np.zeros((rows, cols))
        for r in range(rows):
            list_length = len(obj[r])
            M[r, :] = obj[r] + [obj[r][-1]]*(cols-list_length)
    elif mode == 'cut':
        cols = min([len(obj[k]) for k in range(rows)])
        M = np.zeros((rows, cols))
        for r in range(rows):
            M[r, :] = obj[r][:cols]
        
    return M

def best_so_far(m):
    "Returns best value up to each point along the second axis of a matrix for each row."
    m = np.array(m)
    M = np.zeros_like(m)
    for k in range(M.shape[-1]):
        M[:, k] = np.max(m[:, :k+1], axis=1)
    return M

def plot_exploration(exploration_results, expl_optims=True, highlight_max=False, figsize=(10, 5),
                     fontsize=18, ticksize=14):
    """Plots exploration results. It shows the best bound obtained for every new visited state.
    If `expl_optims=True`, it also shows the proximity to the optimal relaxation."""
    bounds = arrange_shape(exploration_results['bounds'])
    best_bounds = best_so_far(bounds).T
    fig, ax = plt.subplots(figsize=figsize)
    color = "tab:blue"
    ax.plot(best_bounds, color=color, alpha=0.3)
    ax.plot(np.mean(best_bounds, axis=-1), linewidth=4, color=color)
    if highlight_max: ax.plot(np.max(best_bounds, axis=-1), color='C3')
    ax.tick_params(labelsize=ticksize)
    ax.grid()
    ax.set_ylabel("Best obtained bound", fontsize=fontsize)
    ax.set_xlabel("New visited state", fontsize=fontsize)
    
    if expl_optims:
        ax.set_ylabel("Best obtained bound", color=color, fontsize=fontsize)
        ax.tick_params(axis='y', labelcolor=color)
        opt = arrange_shape(exploration_results['oracle_rewards'])
        best_opt = best_so_far(opt).T
        ax2 = ax.twinx()
        color2 = "tab:orange"
        ax2.plot(np.mean(best_opt, axis=-1), color=color2, linewidth=2)
        ax2.set_ylabel("Optimality",color=color2,fontsize=fontsize)
        ax2.tick_params(axis='y', labelcolor=color2, labelsize=ticksize)  
        ax2.set_ylim([-0.05, 1.05])
        
    fig.tight_layout()

In [ ]:
#| export
def convergence_time(results, tol=5e-4, T=50, t_avg=20, return_diffs=False):
    "Returns the convergence with criterion of not changing result by `tol` for `T` epochs."
    if 'training' in results.keys(): results = results['training']
    rewards = arrange_shape(results['rewards'])
    mean_rewards = np.mean(rewards, axis=0)
    epochs = len(mean_rewards)
    moving_avg = np.convolve(np.array([1/t_avg]*t_avg), mean_rewards, mode='valid')
    diffs = np.abs(moving_avg[1:] - moving_avg[:-1])
    diff_variation = np.convolve(np.array([1/T]*T), diffs, mode='valid')
    try:    t = np.where(diff_variation <= tol)[0][0]
    except: t = len(mean_rewards)
    if return_diffs: return t + 2*T, moving_avg, diff_variation 
    return t + T

@delegates(convergence_time)
def indiv_convergence_time(results, max_epochs=800, **kwargs):
    "Similar to `convergence_time` but with each agent."
    results = results['rewards']
    times = [convergence_time({'rewards': [res[:max_epochs]]}, **kwargs) for res in results]
    return np.array(times)

def get_indiv_times(tl_eval, convergence_crit=None):
    "Provides convergence times from individual ratios."
    default_crit = {'T': 50, 't_avg': 100, 'tol': 2e-4}
    convergence_crit = {**default_crit, **convergence_crit} if convergence_crit is not None else default_crit
    Bs = list(TL_evaluation.keys()); Bs.sort()
    time_ratios, time_err = [], []
    for b in Bs:
        ts_tl = indiv_convergence_time(tl_eval[b]['tl'], **convergence_crit)
        ts_0 = indiv_convergence_time(tl_eval[b]['vanilla'], **convergence_crit)
        t0, ttl = ts_0.mean(), ts_tl.mean()
        ratio = ttl/t0
        ratio_std = np.sqrt((1/t0)**2*ts_tl.var() + (ttl/t0**2)**2*ts_0.var())
        time_ratios.append(ratio)
        time_err.append(ratio_std/np.sqrt(len(ts_0)))
    return Bs, np.array(time_ratios), np.array(time_err)

## Load and save data

In [ ]:
#| export
class CPU_Unpickler(pickle.Unpickler):
    "Unpickles from GPU to CPU"
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [ ]:
#| export
bench_dir  = Path("../benchmarks/")
ckp_dir    = Path("../trained_models/checkpoints/")
agents_dir = Path("../trained_models/")

def get_problem_file_name(problem):
    "Returns a consistent file naming from a problem instance."
#     supports = '_'.join([state2str(i[0]) for i in problem.to_sdp()])
    graph = problem.graph.__class__.__name__
    terms = terms = '_'.join([f'{problem._get_weight(i[1]):.2f}' for i in problem.to_sdp()])
    return f"{problem.name}_N{problem.graph.n_nodes}_{graph}_terms_{terms}"

def get_agent_file_name(problem, budget, ID):
    "Returns a consistent file naming for agents."
    return f"agent_{ID}_budget_{budget}_{get_problem_file_name(problem)}"

def get_memory_file_name(problem, solver):
    "Returns a consistent file naming for memories."
    solver_name = re.findall('SdP(.*?)Solver', solver.__class__.__name__)[0]
    return f"memory_{solver_name}_{get_problem_file_name(problem)}"

def get_benchmark_file_name(problem, budget, method):
    "Returns a consistent file naming for benchmarks."
    return f"bench_{method}_budget_{budget}_{get_problem_file_name(problem)}"

def get_checkpoint_file_name(problem, budget, ID, episode):
    "Returns a consistent file naming for checkpoints."
    return f"ckp_{ID}_e_{episode}_budget_{budget}_{get_problem_file_name(problem)}"

def save_model(agent, problem, budget, ID):
    "Saves agent model and parameters."
    agent_name = f"{get_agent_file_name(problem, budget, ID)}.pt" 
    torch.save({'model': agent.model, 'state_dict': agent.model.state_dict}, agents_dir/agent_name)

def load_model(problem, budget, ID):
    "Loads agent model and parameters."
    agent_name = f"{get_agent_file_name(problem, budget, ID)}.pt"        
    return torch.load(agents_dir/agent_name)

def save_benchmark(benchmark, problem, budget, method, suffix=None):
    "Saves benchmark data."
    bench_dir.mkdir(exist_ok=True)
    file_name = get_benchmark_file_name(problem, budget, method)
    if suffix is not None: file_name += f"_{suffix}"
    save_path = (bench_dir/file_name).with_suffix(".pkl")
    with open(save_path, "wb") as f:
        pickle.dump(benchmark, f, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_benchmark(problem, budget, method, suffix=None):
    "Loads benchmark data."
    file_name = get_benchmark_file_name(problem, budget, method)
    if suffix is not None: file_name += f"_{suffix}"
    load_path = (bench_dir/file_name).with_suffix(".pkl")
    with open(load_path, "rb") as f:
        benchmark = pickle.load(f)
    return benchmark

def load_checkpoint(problem, budget, ID, episode):
    ckp_name = f"{get_checkpoint_file_name(problem, budget, ID, episode)}.pt"
    return torch.load(ckp_dir/ckp_name)

def checkpoint2results(N, model, budget=None, IDs=None, episode=None):
    "Loads checkpoints from IDs and converts them to result format."
    models, envs = [], []
    final_rewards, final_costs, final_bounds, final_optims, expl_optims = [], [], [], [], []
    visited_states, visited_bounds, visited_costs = [], [], []
    oracle_rewards, visited_rewards = [], []
    if IDs is None:
        ckp_paths = [c for c in ckp_dir.ls() if f"N{N}" in str(c) and f"{model}" in str(c)]
        if budget is not None:  ckp_paths = [c for c in ckp_paths if f"budget_{budget}_" in str(c)]
        if episode is not None: ckp_paths = [c for c in ckp_paths if f"e_{episode}" in str(c)]
        checkpoints = [torch.load(c) for c in ckp_paths]
    else:
        # Weird function behaviour, I know. For now, I just need to get through this.
        checkpoints = []
        for ID in IDs:
            try:    checkpoints.append(load_checkpoint(N, model, budget, ID, episode))
            except: print(f"Failed to load ID{ID}")
    for ckp in checkpoints:
        models.append(ckp['model']); envs.append(ckp['env']); final_rewards.append(ckp['final_reward'])
        final_costs.append(ckp['final_costs']); final_bounds.append(ckp['final_bounds'])
        final_optims.append(ckp['eval_optims']); expl_optims.append(ckp['expl_optims'])
        visited_states.append(ckp['visited_states']); visited_bounds.append(ckp['visited_bounds'])
        visited_costs.append(ckp['visited_costs']); oracle_rewards.append(ckp['oracle_rewards'])
        visited_rewards.append(ckp['visited_rewards'])
            
    training_results = {'models': models, 'envs': envs, 'rewards': final_rewards,
                        'costs': final_costs, 'bounds': final_bounds, 'eval_optims': final_optims, 
                        'expl_optims': expl_optims}
    exploration_results = {'agents': models, 'envs': envs, 'visited_states': visited_states,
                           'bounds': visited_bounds, 'costs': visited_costs,
                           'oracle_rewards': oracle_rewards, 'visited_rewards': visited_rewards}
    return {'training': training_results, 'exploration': exploration_results}

## State transformations

In [ ]:
#| export
def simplify_layout(L):
    "Simplifies the given layout of the form `[np.array([0, 1]), np.array([0, 1, 2])]`."
    to_simplify = []
    for k1, sites1 in enumerate(L[:-1]):
        if k1 not in to_simplify:
            for k2, sites2 in enumerate(L[k1+1:]):
                overlap = len(np.intersect1d(sites1, sites2))
                if   overlap == len(sites1): to_simplify.append(k1); break
                elif overlap == len(sites2): to_simplify.append(k1+1+k2)
    return [sites for k, sites in enumerate(L) if k not in to_simplify]

def fill_layout(L, N):
    "Fills layout with single-body terms that are missing."
    for n in range(N):
        included = any([(sites == n).any() for sites in L])
        if not included: L.append(np.array([n]))
    return L
    
def state2int(state):
    "Takes state vector as binary encoding of an integer."
    return int(state2str(state[::-1]), 2)

def state2str(state):
    "Takes state vector and outputs a (spaceless) string."
    return ''.join(str(int(i)) if isinstance(i, (int, np.int64)) or i.is_integer() else str(i) for i in state)
    
def T(state):
    "Takes state and converts it to `torch.FloatTensor`"
    return torch.FloatTensor(state)
    
def flip(state, i):
    "Flips constraint `i` from state."
    flip_state = deepcopy(state)
    flip_state[i] = -flip_state[i] + 1
    return flip_state

def add_subgraph_size(subgraphs):
    """Takes a list of subgraphs up to any given size and adds a sublist with all the possible
    subgraphs of a larger size. It assumes the graph edges as minimal graph size."""
    new_subgraphs = []
    for edge in subgraphs[0]:
        for subgraph in subgraphs[-1]:
            if len(np.intersect1d(edge, subgraph)) == 1:
                new_subgraph = np.unique(np.concatenate((edge, subgraph)))
                if not array_in_list(new_subgraph, new_subgraphs):
                    new_subgraphs.append(new_subgraph)
    if len(new_subgraphs) == 0: return subgraphs
    return [*subgraphs, new_subgraphs]

def contained_constraints(state, basis):
    """Returns a mask indicating which constraints in the `state` are contained by larger elements
    of the basis in the same state, excluding themselves. Better to use the
    `SdPEnvironment.contained_constraints` method when possible (it's faster)."""
    contained = np.zeros_like(state, dtype=bool)
    active = basis[state.astype(bool)]
    for i, supp in enumerate(basis):
        for supp_active in active:
            common = np.intersect1d(supp, supp_active) 
            if (len(common) == len(supp)) and (len(common) < len(supp_active)):
                contained[i] = True
                break
    return contained
    
def action_mask(state, basis):
    """Mask of the actions that can be performed. Better to use `SdPEnvironment.action_mask` if
    possible, it's much faster."""
    return np.concatenate(~contained_constraints(state, basis), True)

## Array operations

In [ ]:
#| export
def array_in_list(array, array_list):
    "Checks whether an array is in a list of arrays."
    return any((array == x).all() for x in array_list)

def arrayfy(inp, size):
    "Transforms input into an array of the desired size."
    if isinstance(inp, Iterable): inp = list(inp)
    else:                         inp = [inp]
    return np.array(np.ceil(size/len(inp)).astype(int)*inp)

## Mathematical operations

In [ ]:
#| export     
def dist_exp(xmin, x):
    """Computes an eponential distance given a minimum value (in terms of absolute value)."""
    return np.exp(1-x/xmin)

def dist_poly(x, xmin, xmax, d):
    """Computes a linear decay bound from 1 to 0 given a minimum and maximum value (in terms of 
    absolute value). Exponentiate the line to get better shapes"""
    if np.allclose(xmin, xmax): x[:] = 1.; return x 
    else:                       return ((x-xmax)/(xmin-xmax))**d 
    
def binomial(n, k):
    return np.math.factorial(n)/(np.math.factorial(k)*np.math.factorial(n-k))

#| hide
## Export-

In [ ]:
#| include: false
from nbdev import nbdev_export
nbdev_export()

Converted 00_environment.ipynb.
Converted 01_agents.ipynb.
Converted 02_budget_profiles.ipynb.
Converted 03_hamiltonian.ipynb.
Converted 04_training.ipynb.
Converted 05_utils.ipynb.
Converted 06_sdp.ipynb.
Converted index.ipynb.
